# Retail Store Sales

In this notebook, we will be interacting with a set of provided retail store sails in order to practice various time series forecasting techniques.

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import numpy as np
import pandas as pd

In [2]:
# Setting the filepath for the data
data_filepath = '../data'

In [3]:
# Setting an indicator whether or not to use the sampled data
use_sampled_data = True

# Setting the suffix based on if we are using the sampled data
if use_sampled_data:
    sampled_suffix = '_sampled'
else:
    sampled_suffix = ''

## Getting Started

In [4]:
# Loading in the transactions data
df_transactions = pd.read_csv(f'{data_filepath}/transactions_data{sampled_suffix}.csv')

In [5]:
# Viewing the first few rows of the transactions data
df_transactions.head()

,date,id,item_id,dept_id,cat_id,store_id,state_id
0,2013-01-01 13:41:03,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
1,2013-01-01 07:30:52,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
2,2013-01-01 11:17:38,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
3,2013-01-01 06:07:58,HOBBIES_1_004_TX_2_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_2,TX
4,2013-01-01 21:51:07,HOBBIES_1_004_TX_2_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_2,TX


In [6]:
# Viewing general information about the transactions data
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111221 entries, 0 to 111220
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      111221 non-null  object
 1   id        111221 non-null  object
 2   item_id   111221 non-null  object
 3   dept_id   111221 non-null  object
 4   cat_id    111221 non-null  object
 5   store_id  111221 non-null  object
 6   state_id  111221 non-null  object
dtypes: object(7)
memory usage: 5.9+ MB


In [7]:
# Converting the datatype of the "date" column to a date time
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

In [8]:
# Aggregating the data to a daily level
df_daily_transactions = df_transactions.groupby(by = [pd.Grouper(key = 'date', freq = 'D'), 'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']).size().reset_index().rename(columns = {0: 'sales'})
df_daily_transactions

,date,id,item_id,dept_id,cat_id,store_id,state_id,sales
0,2013-01-01,FOODS_3_714_TX_2_evaluation,FOODS_3_714,FOODS_3,FOODS,TX_2,TX,10
1,2013-01-01,FOODS_3_714_TX_3_evaluation,FOODS_3_714,FOODS_3,FOODS,TX_3,TX,12
2,2013-01-01,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX,3
3,2013-01-01,HOBBIES_1_004_TX_2_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_2,TX,2
4,2013-01-01,HOBBIES_1_004_TX_3_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_3,TX,1
...,...,...,...,...,...,...,...,...
13340,2016-05-22,HOUSEHOLD_1_247_TX_2_evaluation,HOUSEHOLD_1_247,HOUSEHOLD_1,HOUSEHOLD,TX_2,TX,4
13341,2016-05-22,HOUSEHOLD_1_247_TX_3_evaluation,HOUSEHOLD_1_247,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX,3
13342,2016-05-22,HOUSEHOLD_1_266_TX_1_evaluation,HOUSEHOLD_1_266,HOUSEHOLD_1,HOUSEHOLD,TX_1,TX,1
13343,2016-05-22,HOUSEHOLD_1_266_TX_2_evaluation,HOUSEHOLD_1_266,HOUSEHOLD_1,HOUSEHOLD,TX_2,TX,1


In [9]:
# Setting the index based on the date and ID columns
df_daily_transactions.set_index(keys = ['date', 'id'], inplace = True)
df_daily_transactions

item_id      dept_id   
date       id                                                              
2013-01-01 FOODS_3_714_TX_2_evaluation          FOODS_3_714      FOODS_3  \
           FOODS_3_714_TX_3_evaluation          FOODS_3_714      FOODS_3   
           HOBBIES_1_004_TX_1_evaluation      HOBBIES_1_004    HOBBIES_1   
           HOBBIES_1_004_TX_2_evaluation      HOBBIES_1_004    HOBBIES_1   
           HOBBIES_1_004_TX_3_evaluation      HOBBIES_1_004    HOBBIES_1   
...                                                     ...          ...   
2016-05-22 HOUSEHOLD_1_247_TX_2_evaluation  HOUSEHOLD_1_247  HOUSEHOLD_1   
           HOUSEHOLD_1_247_TX_3_evaluation  HOUSEHOLD_1_247  HOUSEHOLD_1   
           HOUSEHOLD_1_266_TX_1_evaluation  HOUSEHOLD_1_266  HOUSEHOLD_1   
           HOUSEHOLD_1_266_TX_2_evaluation  HOUSEHOLD_1_266  HOUSEHOLD_1   
           HOUSEHOLD_1_266_TX_3_evaluation  HOUSEHOLD_1_266  HOUSEHOLD_1   

                                               cat_id store_id state_id  sales  
date       id                                                                   
2013-01-01 FOODS_3_714_TX_2_evaluation          FOODS     TX_2       TX     10  
           FOODS_3_714_TX_3_evaluation          FOODS     TX_3       TX     12  
           HOBBIES_1_004_TX_1_evaluation      HOBBIES     TX_1       TX      3  
           HOBBIES_1_004_TX_2_evaluation      HOBBIES     TX_2       TX      2  
           HOBBIES_1_004_TX_3_evaluation      HOBBIES     TX_3       TX      1  
...                                               ...      ...      ...    ...  
2016-05-22 HOUSEHOLD_1_247_TX_2_evaluation  HOUSEHOLD     TX_2       TX      4  
           HOUSEHOLD_1_247_TX_3_evaluation  HOUSEHOLD     TX_3       TX      3  
           HOUSEHOLD_1_266_TX_1_evaluation  HOUSEHOLD     TX_1       TX      1  
           HOUSEHOLD_1_266_TX_2_evaluation  HOUSEHOLD     TX_2       TX      1  
           HOUSEHOLD_1_266_TX_3_evaluation  HOUSEHOLD     TX_3       TX      1  

[13345 rows x 6 columns]

In [10]:
# Getting the unique dates and IDs from the index of our new dataframe
unique_dates = df_daily_transactions.index.get_level_values('date')
unique_ids = df_daily_transactions.index.get_level_values('id')

# Creating a MultiIndex from the product of the unique dates and IDs
full_index = pd.MultiIndex.from_product([unique_dates, unique_ids], names = ['date', 'id'])

In [11]:
# Re-indexing the data with the new index
df_daily_transactions = df_daily_transactions.reindex(full_index)